# Práctica Python
## Alumnos: _Tania Batista, Fernando de la Fuente, Lourdes Ruiz_
## Tema: Análisis del S&P 500
### Profesor: _Miguel Angel Sicilia_

Hemos elegido para la práctica una tabla de datos que contiene las cotizaciones de 503 valores que componen el índice de bolsa americana S&P 500 de 5 años, desde el 13 de Agosto de 2012 hasta el 11 de Agosto de 2017.
Los datos están extraídos de Kaggle, en el siguiente repositorio:

[Kaggle S&P 500](https://www.kaggle.com/camnugent/sandp500/)

A partir de estos datos, sin tener los datos del índice, sino los de los valores que lo componen, haremos algunos estudios, así como la composición de un índice sintético.
Deicidimos componer un sintético debido a que:
* El índice real varía diariamente el peso de cada uno de los valores que lo componen, y el acceso a esta información que es necesaria para componer el índice real es de pago. 
* Periodicamente se reune un comité que decide qué valores entran y salen del índice, y a lo largo de los 5 años de los que tenemos datos ha habido varios valores que han entrado y otros que han salido. Nuestra base de datos contiene información de los 503 valores que componían el índice en fecha 14 de Agosto de 2017, y sobre esta información es sobre la que trabajaremos.

Por tanto, debido a estas limitaciones no es posible recomponer el índice real y comprobar los datos obtenidos con los de cualquier otra fuente, y en consecuencia fabricaremos uno sintético asumiendo que en el periodo no ha variado la composición, y todos los valores tienen el mismo peso.

Comenzamos por tanto la práctica.

Lo primero sería cargar los datos, y volcarlos a un data frame.

In [ ]:
# PRUEBA PARA ABRIR EL FICHERO DESDE LA URL, ESTA COMPRIMIDO EN ORIGEN
import pandas as pd
import bz2
path ='https://www.kaggle.com/camnugent/sandp500/downloads/all_stocks_5yr.csv.bz2'
sp500 = pd.read_csv(path, delimiter=',',decimal='.', header=0)
sp = pd.DataFrame(sp500)
print(sp.head(10))

In [ ]:
# PRUEBA PARA ABRIR EL FICHERO DESDE LA CARPETA DESCARGAS, ESTA COMPRIMIDO AL DESCARGARLO. --> FUNCIONA
import pandas as pd
path ='C:\\Users\\ffuen\\Downloads\\all_stocks_5yr.csv.zip'
sp500 = pd.read_csv(path, delimiter=',',decimal='.', header=0)
print(sp500.head(10))

In [1]:
# POR EL MOMENTO ESTA ES LA CELDA DE CARGA DE DATOS CORRECTA, DESDE LOCAL.
# ELIMINARLA CUANDO TENGAMOS CARGA DE DATOS DESDE URL

import pandas as pd
import numpy as np
#import csv, operator
path ='C:\\Users\\ffuen\\Desktop\\all_stocks_5yr.csv'
#path ='https://www.kaggle.com//camnugent//sandp500//downloads//all_stocks_5yr.csv'
sp500 = pd.read_csv(path, delimiter=',',decimal='.', header=0, )
sp = pd.DataFrame(sp500)
print(sp.head(10))
print(sp.tail(10))

         Date   Open   High    Low  Close     Volume Name
0  2012-08-13  92.29  92.59  91.74  92.40  2075391.0  MMM
1  2012-08-14  92.36  92.50  92.01  92.30  1843476.0  MMM
2  2012-08-15  92.00  92.74  91.94  92.54  1983395.0  MMM
3  2012-08-16  92.75  93.87  92.21  93.74  3395145.0  MMM
4  2012-08-17  93.93  94.30  93.59  94.24  3069513.0  MMM
5  2012-08-20  94.00  94.17  93.55  93.89  1640008.0  MMM
6  2012-08-21  93.98  94.10  92.99  93.21  2302988.0  MMM
7  2012-08-22  92.56  93.36  92.43  92.68  2463908.0  MMM
8  2012-08-23  92.65  92.68  91.79  91.98  1823757.0  MMM
9  2012-08-24  92.03  92.97  91.94  92.83  1945796.0  MMM
              Date   Open   High    Low  Close     Volume Name
606791  2017-07-31    NaN    NaN  62.32  62.52  2130374.0  ZTS
606792  2017-08-01  62.60  62.69  61.92  62.27  1750974.0  ZTS
606793  2017-08-02  62.04  62.21  61.38  61.54  1831228.0  ZTS
606794  2017-08-03  61.23  62.47  61.20  62.28  1768815.0  ZTS
606795  2017-08-04  62.37  62.85  61.86  62.12 

Comprobamos la cantidad de información que tiene la tabla, en filas y columnas:

In [2]:
print(sp.shape)
print("El número de dimensiones del data frame es de",sp.ndim)
print("El número de filas del data frame es de",len(sp.index))
print(sp.index)
print("El número de columnas del data frame es de",len(sp.columns))
print(sp.columns)
print(type(sp500))

(606801, 7)
El número de dimensiones del data frame es de 2
El número de filas del data frame es de 606801
RangeIndex(start=0, stop=606801, step=1)
El número de columnas del data frame es de 7
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Name'], dtype='object')
<class 'pandas.core.frame.DataFrame'>


Como hemos visto anteriormente, la tabla hacia el final tiene algunos campos vacíos, que se suplen con 'NaN'. 

Es necesario identificarlos, y sustituirlos para limpiar la tabla antes de trabajar con ella:

In [3]:
print(sp.isnull().any())
print("/nEl número de campos vacíos en la columna Date es",sp["Date"].isnull().sum())
print("El número de campos vacíos en la columna Open es",sp["Open"].isnull().sum())
print("El número de campos vacíos en la columna High es",sp["High"].isnull().sum())
print("El número de campos vacíos en la columna Low es",sp["Low"].isnull().sum())
print("El número de campos vacíos en la columna Close es",sp["Close"].isnull().sum())
print("El número de campos vacíos en la columna Volume es",sp["Volume"].isnull().sum())
print("El número de campos vacíos en la columna Name es",sp["Name"].isnull().sum())

Date      False
Open       True
High       True
Low        True
Close     False
Volume     True
Name      False
dtype: bool
/nEl número de campos vacíos en la columna Date es 0
El número de campos vacíos en la columna Open es 384
El número de campos vacíos en la columna High es 208
El número de campos vacíos en la columna Low es 227
El número de campos vacíos en la columna Close es 0
El número de campos vacíos en la columna Volume es 406
El número de campos vacíos en la columna Name es 0


Como vemos, los campos vacíos se encuentran en algunas columnas, no todas. 
Antes de decidir qué hacer con ellos, hagamos un análisis de qué información es la que contiene cada columna:
* _Date:_ Contiene la fecha de la información de cada fila.
* _Open:_ Es el primer precio al que abrió el valor ese día.
* _High:_ El precio más alto negociado en el día.
* _Low:_ El precio más bajo negociado en el día.
* _Close:_ Indica a qué precio cerró el valor ese día. Este podemos decir que es el dato más importante de la fila.
* _Volume:_ Indica el número de acciones negociadas en el día. No todas cambian de manos al mismo precio.
* _Name:_ Indica el nombre del valor. Es una abreviatura utilizada para identificar cada valor. Por ejemplo: Amazon sería AMZN.

In [4]:
print("La columna Name contiene",len(sp.Name.unique()),"valores distintos.")

print("Son las compañías que formaban el índice S&P en fecha 11 de Agosto de 2017.")

La columna Name contiene 503 valores distintos.
Son las compañías que formaban el índice S&P en fecha 11 de Agosto de 2017.


Vamos a sustituir los NaN que aparezcan en las columnas High y Low por un valor, que será:
* Para la columna _High_ el resultado de incrementar el precio de cierre de ese día por la diferencia (en %) entre el High y el Close --> High = Close * ( 1 + (% diferencia media entre High y Close de todo el periodo)
* Para la columna _Low_ el resultado de disminuir el precio de cierre de ese día por la diferencia (en %) entre el Low y el Close --> Low = Close * ( 1 - (% diferencia media entre el Close y el Low de todo el periodo)

Sustituiremos primero los datos de la columna _High_

In [72]:
#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL HIGH, y SUSTITUIMOS LOS NAN POR ESTE CALCULO

def calculo_reemplazo_high(name):
    subset = sp[sp['Name'] == name]
    subset_high = subset['High']
    subset_close = subset['Close']
    media = np.mean( (subset_high - subset_close) / subset_close) # --> Esta es la fórmula que calcula el promedio)
    return media

# Recorremos los campos que tienen algún NaN en la columna High (son 208), y los sustituimos.

print("El número de campos vacíos en la columna High antes de la sustitución es",sp['High'].isnull().sum())

for posiciones in sp['High'].index[sp['High'].apply(np.isnan)]:
    sp.loc[posiciones,'High'] = sp['Close'][posiciones] * (1 + calculo_reemplazo_high(sp['Name'][posiciones]))
    
print("El número de campos vacíos en la columna High después de la sustitución es",sp['High'].isnull().sum())

El número de campos vacíos en la columna High antes de la sustitución es 208
El número de campos vacíos en la columna High después de la sustitución es 0


Y ahora sustituiremos los datos de _Low_

In [73]:
#OBTENER PROMEDIO, POR CADA VALOR, DE LA DIFERENCIA EN % ENTRE EL CIERRE Y EL LOW, y SUSTITUIMOS LOS NAN POR ESTE CALCULO

def calculo_reemplazo_low(name):
    subset = sp[sp['Name'] == name]
    subset_low = subset['Low']
    subset_close = subset['Close']
    media = np.mean( (subset_low - subset_close) / subset_close) # --> Esta es la fórmula que calcula el promedio
    return media

# Recorremos los campos que tienen algún NaN en la columna Low (son 227), y los sustituimos.

print("El número de campos vacíos en la columna Low antes de la sustitución es",sp['Low'].isnull().sum())

for posiciones in sp['Low'].index[sp['Low'].apply(np.isnan)]:
    sp.loc[posiciones,'Low'] = sp['Close'][posiciones] * (1 + calculo_reemplazo_low(sp['Name'][posiciones]))
    
print("El número de campos vacíos en la columna Low después de la sustitución es",sp['Low'].isnull().sum())

El número de campos vacíos en la columna Low antes de la sustitución es 227
El número de campos vacíos en la columna Low después de la sustitución es 0


A continuación vamos a sustituir los NaN que haya en al columna _Open_, por un número aleatorio que esté entre el valor de la columna _High_ y _Low_ de esa sesión.

In [74]:
# SUSTITUCION DE NaN EN COLUMNA Open, con un aleatorio que esté en el rango entre el High y el Low

print("El número de campos vacíos en la columna Open antes de la sustitución es",sp['Open'].isnull().sum())

for posiciones in sp['Open'].index[sp['Open'].apply(np.isnan)]: 
    sp.loc[posiciones,'Open'] = np.random.uniform (sp['Low'][posiciones], sp['High'][posiciones])

print("El número de campos vacíos en la columna Open después de la sustitución es",sp['Open'].isnull().sum())

El número de campos vacíos en la columna Open antes de la sustitución es 384
El número de campos vacíos en la columna Open después de la sustitución es 0


Respecto a la columna _Volume_, para cada valor que tenga NaN (aunque solo es un único valor) calcularemos la media del volumen de títulos que se negocian diariamente, y sustituiremos el NaN por esta media.

In [75]:
print("El número de campos vacíos en toda la tabla en la columna Volume es",sp["Volume"].isnull().sum())

a = sp['Volume'].index[sp['Volume'].apply(np.isnan)]
a = np.unique(sp.loc[a[0:-1],'Name'])
print("Los valores que tienen algún NaN en la columna Volume son: ",a)

for valores in a:
    sp[sp['Name']==valores] = sp[sp['Name']== valores].fillna(sp[sp['Name']== valores].mean())

print("El número de campos vacíos en la columna Volume es",sp["Volume"].isnull().sum())

El número de campos vacíos en toda la tabla en la columna Volume es 406
Los valores que tienen algún NaN en la columna Volume son:  ['GOOG']
El número de campos vacíos en la columna Volume es 0


Una vez hechos los reemplazos de los datos nulos en las columnas, comprobamos que no quede ninguno:

In [76]:
print(sp.isnull().any())

Date      False
Open      False
High      False
Low       False
Close     False
Volume    False
Name      False
dtype: bool


A partir de la información que tenemos ahora, datos diarios de 503 valores que componen el índice S&P 500, vamos a componer un índice nosotros, similar al original, para poder centrar parte de nuestro análisis en él.

### CONSIDERACIONES PRELIMINARES SOBRE EL ÍNDICE S&P 500
El índice S&P 500 es el índice más representativo de la evolución del mercado norteamericano, y recoge la evolución de las 500 mayores empresas cotizadas en los distintos mercados de renta variable del país. Hoy en día está formado por 503 valores, dado que algunos de ellos tienen distintas clases de acciones con diferentes derechos cotizando.
Un comité es el encargado de la selección de valores que lo componen, siguiendo 8 criterios, que son: capitalización bursatil, liquidez, domicilio, capital flotante, clasificación del sector, viabilidad financiera, periodo de tiempo durante el cual ha cotizado en bolsa y ser negociada en la bolsa de valores.

El comité se reune periódicamente y decide los valores que entran y salen del índice. Respecto a la ponderación de cada valor dentro del índice, esta depende de la capitalización de mercado de cada valor (precio diario X número de acciones en circulación). 

No disponemos en el dataframe del número de acciones que tiene en circulación cada compañía que forma el índice, y por tanto, **no nos es posible construir una reproducción fiel al original del índice S&P 500**. Aunque tuviésemos esta información, durante los 5 años de datos que tenemos han entrado y salido valores del índice, y no disponemos de los datos diarios de los que han salido. 

No obstante, podemos construir un índice sintético alterando las ponderaciones de los valores, utilizando el método *'EQUALLY WEIGHTED'*, en el que todos los valores tienen el mismo peso dentro del índice.

### CONSTRUCCIÓN DEL ÍNDICE
Para crear el indice necesitamos 2 cosas:
* *Un valor inicial* --> Tomaremos como referencia el valor del S&P el 10 de Agosto de 2012, un día hábil antes de la fecha a partir de la cual tenemos datos. Ese día el S&P 500 cerró a 1.405,87 y será el valor a partir del cual partamos.
* *Variaciones diarias del índice* --> A partir de las variaciones diarias de los 504 valores haremos que el índice se mueva diariamente.

Lo primero que tenemos que hacer entonces, es crear una fila para cada día con información del índice, que estará en varias columnas:
* *Date:* fecha en la que se recoge la información
* *Open:* precio en el que abre el índice. Carece de sentido real, ya que los precios a los que abre cada valor no tienen por qué coincidir en el tiempo (unos pueden cruzar operaciones nada más abrir el mercado, pero otros pueden hacerlo más tarde).
* *High:* precio más alto alcanzado por el índice. Carece también de sentido real, ya que los High de cada valor se producen en diferentes momentos durante la sesión.
* *Low:* precio más bajo alcanzado por el índice. Carece también de sentido real, ya que los Low de cada valor se producen en diferentes momentos durante la sesión.
* *Close:* precio al que cierra el índice. Es el dato más importante, y se construye a partir de los cierres de los 504 valores.

Pero antes de eso, debemos ordenar la tabla por valor y luego por fecha, para posteriormente añadir una columna a nuestro dataframe que recoja la variación diaria de cada valor:

In [95]:
sp['Change'] = np.zeros(sp.index.size)
for valor in lista_valores:

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
         ... 
606771    0.0
606772    0.0
606773    0.0
606774    0.0
606775    0.0
606776    0.0
606777    0.0
606778    0.0
606779    0.0
606780    0.0
606781    0.0
606782    0.0
606783    0.0
606784    0.0
606785    0.0
606786    0.0
606787    0.0
606788    0.0
606789    0.0
606790    0.0
606791    0.0
606792    0.0
606793    0.0
606794    0.0
606795    0.0
606796    0.0
606797    0.0
606798    0.0
606799    0.0
606800    0.0
Name: Change, Length: 606801, dtype: float64